# Part_D : Attribute Reduction by Principal Component Analysis

In [1]:
import pandas as pd
import numpy as np
import os
df=pd.read_csv('processed_dataset.csv')
df.head()

foolishness  hath  wholesome  takest  feelings  anger  vaivaswata  matrix  \
0          0.0   0.0        0.0     0.0       0.0    0.0         0.0     0.0   
1          0.0   0.0        0.0     0.0       0.0    0.0         0.0     0.0   
2          0.0   0.0        0.0     0.0       0.0    0.0         0.0     0.0   
3          0.0   0.0        0.0     0.0       0.0    0.0         0.0     0.0   
4          0.0   0.0        0.0     0.0       0.0    0.0         0.0     0.0   

   kindled  convict  ...  thinkest  modern  reigned  sparingly  visual  \
0      0.0      0.0  ...       0.0     0.0      0.0        0.0     0.0   
1      0.0      0.0  ...       0.0     0.0      0.0        0.0     0.0   
2      0.0      0.0  ...       0.0     0.0      0.0        0.0     0.0   
3      0.0      0.0  ...       0.0     0.0      0.0        0.0     0.0   
4      0.0      0.0  ...       0.0     0.0      0.0        0.0     0.0   

   thoughts  illumines  attire  explains  output_label  
0       0.0        0.0     0.0       0.0      Buddhism  
1       0.0        0.0     0.0       0.0      Buddhism  
2       0.0        0.0     0.0       0.0      Buddhism  
3       0.0        0.0     0.0       0.0      Buddhism  
4       0.0        0.0     0.0       0.0      Buddhism  

[5 rows x 8267 columns]

In [2]:
# for array of values
df_array=df.drop(['output_label'],axis=1).values
# for prestored cosine similarity values
dot_array=np.dot(df_array,df_array.T)

In [3]:
from sklearn.decomposition import PCA

In [4]:
#applying Principal Component Analysis
pca=PCA(n_components=100)
df_array_pca=pca.fit_transform(df_array)
dot_array_pca=np.dot(df_array_pca,df_array_pca.T)

In [26]:

# m - number of training examples
# n - number of features in each training example


class qnode:
    def __init__(self,data_point_index_1,data_point_index_2,dist):
        self.data_point_index_1=data_point_index_1
        self.data_point_index_2=data_point_index_2
        self.dist=dist

class disjoint_sets:

    def __init__(self,m):
        self.parent=[]
        self.rank=[]
        for i in range(m): 
            self.parent.append(i)
            self.rank.append(0)


    def find_parent(self,i):
        if self.parent[i]==i: return i
        else:
            p=self.find_parent(self.parent[i])
            self.parent[i]=p
            return p

    def union(self,i,j):
        p_i=self.find_parent(i)
        p_j=self.find_parent(j)
        if(p_i==p_j) : return False
        else:
            if self.rank[p_i]<self.rank[p_j]: self.parent[p_i]=p_j
            elif self.rank[p_i]>self.rank[p_j]: self.parent[p_j]=p_i
            else:
                self.parent[p_i]=p_j
                self.rank[p_j]+=1
            return True  


def merge(A,p,q,r):
    L=[]
    R=[]
    for i in range(p,q+1): L.append(A[i])
    for j in range(q+1,r+1): R.append(A[j])
    n1=len(L)
    n2=len(R)
    assert(n1==q-p+1)
    assert(n2==r-q)
    k=p
    i=0
    j=0
    while i<n1 and j<n2:
        if L[i].dist<=R[j].dist:
            A[k]=L[i]
            i+=1
        else:
            A[k]=R[j]
            j+=1
        k+=1
    while i<n1:
        A[k]=L[i]
        k+=1
        i+=1
    while j<n2:
        A[k]=R[j]
        k+=1
        j+=1

def mergeSort(A,p,r):
    if p<r:
        q=(p+r)//2
        mergeSort(A,p,q)
        mergeSort(A,q+1,r)
        merge(A,p,q,r)





path=os.getcwd()
parent=os.path.dirname(path)

X=dot_array_pca

dist=np.exp(-(X @ X.T))


L=[]
for i in range(X.shape[0]):
    for j in range(i+1,X.shape[0]):
        L.append(qnode(i,j,dist[i,j]))
mergeSort(L,0,len(L)-1)


m,n=X.shape

clusters=disjoint_sets(m)

num_clusters=m
k=0
while num_clusters!=8 and k<len(L):
    node=L[k]
    i1=node.data_point_index_1
    i2=node.data_point_index_2	
    if clusters.union(i1,i2): num_clusters-=1
    k+=1

cluster_parents=set()
for i in range(m):
     cluster_parents.add(clusters.find_parent(i))

i=0
d=dict()
for c in cluster_parents:
    d[c]=i
    i+=1

cluster_labels=[[] for i in range(num_clusters)]
for i in range(m):
    p=clusters.find_parent(i)
    p=d[p]
    cluster_labels[p].append(i)

cluster_labels.sort()
for c in cluster_labels:
    print(c)
    print()

f=open('agglomerative_reduced.txt','w+')
#writing the 8 created clusters into the file 'agglomerative.txt'
for i in range(8):
    cluster_labels[i].sort()
    for j in range(len(cluster_labels[i])):
        f.write(str(cluster_labels[i][j]))
        if j!=len(cluster_labels[i])-1:
            f.write(',')
    f.write('\n')    
f.close()




[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 142, 143, 144, 145, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226

In [27]:
num_clusters_required=8 # REQUIRED NO. OF CLUSTERS DECLARED AT THE BEGINNING

In [28]:
def vector_distance(v1,v2): # v1,v2: unit l2-normalised vectors, FUNCTION RETURNS DISTANCE BETWEEN THE VECTORS
    cosine_sim=np.dot(v1,v2)
    return np.exp(-cosine_sim)

In [29]:
# RETURN CLUSTERS OF DOCUMENTS GIVEN CENTROIDS OF CLUSTERS
def assign_clusters(tfidf_mat,centroids,cluster_assignment,num_docs):
    f=0 # f=0 denotes no change from previous cluster assignment, so algorithm terminates, else it continues (f=1)
    cluster_docs={}
    for i in range(num_docs):
        doc_vector=tfidf_mat[i]
        min_dist=-1
        for j in range(num_clusters_required):
            centroid_vector=centroids[j]
            dist=vector_distance(doc_vector,centroid_vector) # DISTANCE OF DOCUMENT I FROM CLUSTER J
            if min_dist<0:
                min_dist=dist
                closest_centroid=j #UPDATE CLOSEST CLUSTER CENTROID
                continue
            else:
                if min_dist>dist:
                    min_dist=dist
                    closest_centroid=j #UPDATE CLOSEST CLUSTER CENTROID
        if closest_centroid not in cluster_docs.keys(): #IF THE CLUSTER INDEX EXISTS IN KEY LIST
            cluster_docs[closest_centroid]=[i]
        else:
            cluster_docs[closest_centroid].append(i)
        if cluster_assignment[i]!=closest_centroid: # CHANGE FROM PREVIOUS ASSIGNMENT OF DOC TO CLUSTER, ASSIGNED TO SOME OTHER CLUSTER, SO f=1
            cluster_assignment[i]=closest_centroid
            f=1
    return f,cluster_docs

In [30]:
def update_centroids(cluster_docs,centroids,tfidf_mat):
    for key in sorted(cluster_docs):
        centroids[key]=np.mean(tfidf_mat[cluster_docs[key]],axis=0) # CENTROID IS THE MEAN ACROSS ALL FEATURES(ACROSS COLUMNS)
    return centroids

In [31]:
# KMEANS CLUSTERING ALGORITHM
def kmeans(tfidf_mat,num_clusters_required):
    num_docs=tfidf_mat.shape[0]
    num_features=tfidf_mat.shape[1]
    centroids=np.random.rand(num_clusters_required,num_features) # CENTROIDS RANDOMLY INITIALISED
    cluster_assignment=[] #FIRST, CLUSTER ASSIGNMENT HAS TO BE DONE
    cluster_docs={} # CONTAINS DOCUMENT INDICES OF EVERY CLUSTER
    for i in range(num_docs):
        doc_vector=tfidf_mat[i]
        min_dist=-1
        for j in range(num_clusters_required):
            centroid_vector=centroids[j]
            dist=vector_distance(doc_vector,centroid_vector)
            if min_dist<0:
                min_dist=dist
                closest_centroid=j
                continue
            else:
                if min_dist>dist:
                    min_dist=dist
                    closest_centroid=j
        cluster_assignment.append(closest_centroid)
        if closest_centroid not in cluster_docs.keys():
            cluster_docs[closest_centroid]=[i]
        else:
            cluster_docs[closest_centroid].append(i)
    f=0
    iterations=0
    while iterations<10000: # REPEAT UNTIL CONVERGENCE OF CLUSTERS
        centroids=update_centroids(cluster_docs,centroids,tfidf_mat) # M-STEP (CENTROID UPDATE)
        f,cluster_docs=assign_clusters(tfidf_mat,centroids,cluster_assignment,num_docs) # E-STEP (CLUSTER ASSIGNMENT)
        if f==0: # No change in cluster assignments, algorithm terminates
            break
        iterations+=1
    ordered_cluster_docs=sorted(cluster_docs.items(),key = lambda x : x[1]) #SORT CLUSTERS BY INDICES OF THEIR COMPONENT DOCUMENTS
    return ordered_cluster_docs

In [32]:
def write(ordered_cluster_docs,filename): # filename, including extension .txt, WRITE THE CLUSTER INFORMATION TO FILE
    c=0
    with open('%s'%(filename),'w') as f:
        for item in ordered_cluster_docs:
            f.write(','.join([str(idx) for idx in item[1]]))
            c+=1
            if c<num_clusters_required:
                f.write('\n')

In [33]:
tfidf_mat_pca=df_array_pca
ordered_cluster_docs=kmeans(tfidf_mat_pca,num_clusters_required) # GET ORDERED CLUSTERS FROM KMEANS
write(ordered_cluster_docs,'kmeans_reduced.txt') # WRITE CLUSTERS TO FILE

In [24]:
len(ordered_cluster_docs)

8

In [25]:
for c in range(8):
    print(ordered_cluster_docs[c])

(4, [0, 33, 126, 127, 128, 129, 131, 132, 133, 142, 190, 191, 192, 194, 195, 197, 198, 200, 203, 205, 206, 207, 211, 212, 214, 216, 222, 223, 225, 234, 235, 236, 244, 245, 246, 249, 255, 351, 370, 377, 378, 444, 452, 461, 464])
(6, [1, 2, 3, 7, 8, 9, 11, 13, 15, 16, 19, 20, 21, 22, 27, 28, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 42, 43, 306, 385, 446])
(2, [4, 12, 17, 18, 23, 24, 25, 26, 41, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 130, 134, 135, 137, 138, 140, 143, 144, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 160, 162, 163, 164, 165, 166, 168, 169, 170, 171, 172, 173, 175, 180, 182, 184, 186, 188, 189, 199, 201, 202, 210, 215, 217, 218, 219, 220, 224, 226, 227, 228, 229, 230, 